In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.7 MB/s eta 0:00:00


In [47]:
!pip install datasets transformers[sentencepiece] sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
from tqdm import tqdm

In [49]:
from datasets import load_metric
metric = load_metric("sacrebleu")

<ipython-input-49-aede95515358>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [4]:
# Load the dataset
data = pd.read_csv('most_toxic_data.csv').head(10000)

In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU available, using the CPU instead.")


There are 1 GPU(s) available.
Device name: Tesla T4


In [6]:
# Define the Dataset
class DetoxDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.input_texts = data['reference'].tolist()  # Assuming this is the toxic text
        self.target_texts = data['translation'].tolist()  # Assuming this is the non-toxic text
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        source_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        # Tokenize the source text
        source = self.tokenizer.batch_encode_plus(
            [source_text], max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # Tokenize the target text
        target = self.tokenizer.batch_encode_plus(
            [target_text], max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        source_ids = source['input_ids'].squeeze()
        target_ids = target['input_ids'].squeeze()

        # The labels are the target_ids without the padding token
        labels = target_ids.masked_fill(target_ids == self.tokenizer.pad_token_id, -100)

        # No need to move to device here, as this will be handled by the data loader and Trainer later
        return {"input_ids": source_ids, "attention_mask": source["attention_mask"].squeeze(), "labels": labels}

In [7]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

# Create the dataset
dataset = DetoxDataset(tokenizer, data)

# Split the dataset
train_size = int(0.8 * len(dataset))
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [9]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,2.412800,1.952261
2,2.235700,1.880274
3,2.029800,1.850349
4,1.974300,1.833162
5,1.917000,1.824938
6,1.674400,1.821040
7,1.764600,1.820155
8,1.461700,1.819587
9,1.823900,1.818088
10,1.601700,1.817509


TrainOutput(global_step=20000, training_loss=1.8637295840740205, metrics={'train_runtime': 7554.4431, 'train_samples_per_second': 10.59, 'train_steps_per_second': 2.647, 'total_flos': 1.082734411776e+16, 'train_loss': 1.8637295840740205, 'epoch': 10.0})

In [10]:
# Save the model
model.save_pretrained('detoxified_t5_model')
tokenizer.save_pretrained('detoxified_t5_model')

('detoxified_t5_model/tokenizer_config.json',
 'detoxified_t5_model/special_tokens_map.json',
 'detoxified_t5_model/spiece.model',
 'detoxified_t5_model/added_tokens.json')

In [ ]:
model_path = 'detoxified_t5_model'
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Function to use the fine-tuned model for prediction
def detoxify_text(text):
    model.eval()
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)  # "detoxify:" is the task prefix
    outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

In [43]:
def is_toxic(text_to_analyze, comments = False):
  # Run the classifier (model and tokenizer)
  results = classifier(text_to_analyze)

  # Interpret the results
  for result in results:
      label = result['label']
      score = result['score']


      # Heuristic for toxicity based on negative sentiment score
      res = label == 'NEGATIVE' and score > 0.98
      if comments:
        # Print results
        print(f"Label: {label}, Score: {score}")
        if res:
            print("The text may be considered toxic.")
        else:
            print("The text is unlikely to be toxic.")
      return res

In [44]:
# Example text
toxic_text = "i'm fucking like that shit"
text_to_analyze =detoxify_text(toxic_text)
print(text_to_analyze)
print("Is toxic:", is_toxic( text_to_analyze, True))

i'm like that
Label: POSITIVE, Score: 0.9998190999031067
The text is unlikely to be toxic.
Is toxic: False


In [45]:
import random
data_test = pd.read_csv('test.csv')
test_texts = data_test['reference'].tolist()
random.shuffle(test_texts)
test_texts = test_texts[:1000]

In [46]:
not_toxic = 0

for toxic_text in tqdm(test_texts):
    text_to_analyze =detoxify_text(toxic_text)
    toxity = is_toxic(text_to_analyze, False)
    if not toxity:
      not_toxic += 1
print("\nAccuracy:", not_toxic/len(test_texts))

100%|██████████| 1000/1000 [03:47<00:00,  4.40it/s]


Accuracy: 0.468


In [55]:
data_test = pd.read_csv('test.csv').head(100)
test_texts_reference = data_test['reference'].tolist()
test_texts_translation = [[data] for data in data_test['translation'].tolist()]

predictions = [detoxify_text(toxic_text) for toxic_text in test_texts_reference]
metric.compute(predictions = predictions, references=test_texts_translation)


{'score': 24.47967691937549,
 'counts': [532, 275, 147, 82],
 'totals': [985, 885, 786, 691],
 'precisions': [54.01015228426396,
  31.073446327683616,
  18.702290076335878,
  11.866859623733719],
 'bp': 0.9909045602112877,
 'sys_len': 985,
 'ref_len': 994}